In [1]:
!pip install -q nest_asyncio
!pip install -q -U python-telegram-bot
!pip install -q sentence-transformers pypdf requests pdfminer.six psutil pymupdf
!pip install -q langchain-community langchain-text-splitters faiss-cpu
!pip install -q hf_xet omegaconf

^C


In [ ]:
import os
os.environ["TG_BOT_TOKEN"] = ""
os.environ["OLLAMA_API_KEY"] = ""  
os.environ["OLLAMA_API_URL"] = "https://ollama.com/api/chat"
os.environ["OLLAMA_MODEL"] = "gpt-oss:20b-cloud"


In [ ]:
# PROMPTS / STYLE (используются ботом ниже)

TELEGRAM_STYLE_SYSTEM = (
    "Отвечай для чата Telegram. Вывод только простым текстом. "
    "Запрещено: схемы/ASCII-арт, таблицы, списки с маркерами, Markdown (**, #, ```), JSON/XML/YAML, код. "
    "Пиши кратко и по делу: 6–10 предложений, без лишней воды. "
    "Если информации недостаточно — прямо скажи, чего нет в контексте, и что нужно уточнить."
)

PC_SHEET_PROMPT = """Ты — мастер по созданию персонажей Dungeons & Dragons 5e. На основе предоставленных данных сформируй полное, живое и удобочитаемое описание персонажа, ориентируясь на официальный лист персонажа (character sheet).

Вывод должен быть только в виде простого текста, без форматирования, таблиц, списков или технических конструкций. Каждая секция начинается с заголовка, за которым следует содержание. Соблюдай строго следующий порядок:

Имя: [полное имя]
Раса: [раса и подраса, если есть]
Класс: [класс, архетип/подкласс, уровень]
Предыстория: [происхождение, например: «наёмник», «учёный», «изгнанник»]
Мировоззрение: [например: «нейтрально-добрый»]

Характеристики:
Сила [число] (+модификатор)
Ловкость [число] (+модификатор)
Телосложение [число] (+модификатор)
Интеллект [число] (+модификатор)
Мудрость [число] (+модификатор)
Харизма [число] (+модификатор)

Классовые спасброски: [например: «Сила +5, Телосложение +4»]
Броня: [класс брони, тип брони]
Инициатива: [+модификатор]
Скорость: [в футах, например: «30 фт»]
Максимум хитов: [число]
Текущие хиты: [число]
Временные хиты: [число или «нет»]

Навыки: [перечисли только те, в которых персонаж компетентен, с бонусами, например: «Внимательность +6, Запугивание +5, Акробатика +4»]

Языки и владения: [языки, инструменты, оружие, броня — кратко через запятую]

Черты характера: [1–2 яркие особенности поведения]
Идеалы: [во что верит]
Привязанности: [кто или что ему дорого]
Слабости: [страхи, пороки, внутренние конфликты]

Внешность: [рост, вес, цвет глаз/волос/кожи, одежда, шрамы, украшения — всё, что видно сразу]

Союзники и организации: [кто на его стороне, членство в группах]

Цель: [почему он путешествует, что ищет или защищает]

Особые способности и черты расы/класса: [кратко — только то, что важно для игры и отыгрыша, например: «Темное зрение 60 фт», «Боевой стиль: Защита», «Удача полурослика»]

Не добавляй пояснений, комментариев или лишних слов. Не используй жирный шрифт, курсив, звёздочки, тире как маркеры, JSON, XML или любые другие форматы. Только чистый текст — каждая строка начинается с указанного заголовка."""

NPC_PROMPT = """Создай описание неигрового персонажа (NPC) для Dungeons & Dragons 5e. Выведи только простой текст, без таблиц, списков или форматирования. Соблюдай следующую структуру — каждая строка начинается с заголовка:

Имя:
Раса:
Возраст:
Род занятий:
Мировоззрение:
Внешность:
Характер:
Цель или мотивация:
Отношение к игрокам:
Особенности:
Секрет:

Пиши кратко и естественно. Особенности — это то, что делает NPC запоминающимся (привычка, манера речи, предмет одежды и т.п.). Секрет — может быть незначительным («боится крыс») или важным («контрабандист»), но если NPC действительно обыденный — секрет может быть пустым или бытовым. Не выдумывай эпические детали, если они не уместны. Главное — чтобы было достаточно для живого отыгрыша за 1–2 минуты."""

NPC_DIALOG_PROMPT = """Ты — NPC в фэнтезийной кампании Dungeons & Dragons 5e.

Тебе будет дано:
1) Контекст сеттинга/кампании (если найден), если нет его, используй Фаэрун как основной сеттинг
2) Реплика/ситуация, на которую нужно симмулировать, чаще всего это реплика обращеная к персонажам игроков

Требования к стилю и сеттингу:
Не будь излишне формальным, веди себя естественно
Пиши по-русски, грамотно и связно, как живой персонаж.
Одна цель: выдать правдоподобную реплику NPC.
Пиши как персонаж в средневековом/фэнтезийном мире.
Используй термины D&D или близкие к средневековью (трактир, стража, гильдия, храм, маг, жрец, свиток, арбалет и т.п.).
Запрещены современные термины и концепции: компьютеры, интернет, смартфоны, лазеры, ракеты, электричество в современном смысле, огнестрел как в современном мире.
Запрещён современный сленг и мемы.
Если игрок говорит современными словами, перефразируй их «внутрисеттингово».

Вывод:
Одна реплика (3–5 предложения) от имени NPC/Персонажа. Не забывай использовать междометия, говори калоритно. Без пояснений и без служебных пометок."""

COMBAT_PROMPT = """Ты — опытный Мастер Подземелий, специализирующийся на создании сбалансированных и захватывающих боёв для D&D 5e. На основе входных данных (уровни персонажей, их количество и классы) создай подробное описание боевой встречи.

Выведи ТОЛЬКО простой текст в следующей структуре — каждая секция начинается с заголовка, за которым следует содержание. Никаких таблиц, JSON, markdown или лишних комментариев:

Сложность встречи: [уровень сложности: лёгкая/средняя/сложная/смертельная]
Вероятность победы группы: [примерный процент и краткое обоснование]

ОПИСАНИЕ ВСТРЕЧИ:
[Краткое описание противников: кто они, почему атакуют, их тактика в 1-2 предложениях]

ПРОТИВНИКИ:
[Для каждого типа противника укажи:]

Название: [раса/класс]
Количество: [число]
Класс доспеха: [число]
Хиты: [число + кости хитов, например: "22 (3к8+9)"]
Скорость: [число футов]
Ключевые атаки:
[Название атаки]: [+к_атаке] к атаке, [урон+кости], тип урона
Особые способности: [названия и краткое описание]
Тактические приёмы: [2-3 интересных хода в бою, неочевидные применения способностей]

ОКРУЖЕНИЕ:
[Описание локации: препятствия, укрытия, ловушки, взаимодействующие объекты]

НАГРАДЫ:
Опыт: [общее число опыта для разделения между игроками]
Сокровища: [предметы, золото, магические предметы]
Особые награды: [информация, союзники, преимущества для следующих квестов]

СОВЕТЫ МАСТЕРУ:

Баланс: [что добавить/убрать, если бой слишком лёгкий или сложный]
Атмосфера: [как описать начало боя и ключевые моменты]
Альтернативы: [варианты завершения боя помимо убийства всех противников]
Важные проверки: [какие проверки навыков изменят ход боя]

ДОПОЛНИТЕЛЬНО:
[Полезные примечания: возможные последствия боя, связь с сюжетом, реакция NPC после боя]"""

PLAYER_RULES_PROMPT = """Ты — помощник игрока D&D 5e. Ответь на вопрос по правилам и механикам.

Тебе будет дано:
1) Вопрос игрока
2) (Опционально) фрагменты из базы знаний игрока

Если фрагменты есть — опирайся на них. Если фрагментов нет — ответь по общим знаниям D&D 5e.

Ограничения:
Не используй заголовки капсом.
Не пиши служебные флаги (например, "found", "флаг", "контекст", "база: да/нет").
Не придумывай детали кампании.

Формат вывода:
Сначала короткий ответ (1–2 предложения).
Потом пустая строка.
Потом подробное объяснение (6–10 предложений).
Потом (если уместно) пустая строка и строка "Пример в игре: ...".
"""

PLAYER_SITUATION_PROMPT = """Ты — помощник игрока D&D 5e. Помоги разобраться в ситуации за столом.

Тебе будет дано:
1) Описание ситуации игроком
2) (Опционально) фрагменты из базы знаний игрока

Если фрагменты есть — опирайся на них. Если фрагментов нет — помоги по общим знаниям D&D 5e.

Ограничения:
Не используй заголовки капсом.
Не пиши служебные флаги (например, "found", "флаг", "контекст", "база: да/нет").
Не придумывай детали кампании.

Формат вывода:
Сначала короткая рекомендация (1–2 предложения).
Потом пустая строка.
Потом разбор по правилам и практический совет (8–12 предложений).
Потом (если уместно) пустая строка и строка "Пример в игре: ...".
"""

In [ ]:
# PLAYER FAISS RAG (отдельная база, без доступа к данным DM)
import os
import re
import gc
import logging
from pathlib import Path
from typing import Optional, Tuple, List, Iterator

logger = logging.getLogger("rag-dnd-bot-faiss")

PLAYER_DATASET_DIR = Path(os.getenv("PLAYER_DATASET_DIR", "Dataset/Main"))
PLAYER_FAISS_DIR = Path(os.getenv("PLAYER_FAISS_DIR", "artifacts/faiss_player_knowledge"))
PLAYER_FALLBACK_EMBED_MODEL = os.getenv("PLAYER_FALLBACK_EMBED_MODEL", "sentence-transformers/paraphrase-multilingual-mpnet-base-v2")
PLAYER_EMBED_MODEL = os.getenv("PLAYER_EMBED_MODEL", PLAYER_FALLBACK_EMBED_MODEL)
PLAYER_CHUNK_SIZE = int(os.getenv("PLAYER_CHUNK_SIZE", "2000"))
PLAYER_CHUNK_OVERLAP = int(os.getenv("PLAYER_CHUNK_OVERLAP", "500"))
PLAYER_TOP_K = int(os.getenv("PLAYER_TOP_K", "15"))
PLAYER_STREAM_FILE_BYTES = int(os.getenv("PLAYER_STREAM_FILE_BYTES", str(5 * 1024 * 1024)))


def _normalize_text_player(text: str) -> str:
    if not text:
        return ""
    text = text.replace("\r\n", "\n").replace("\r", "\n")
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


def _iter_char_chunks_from_stream(stream, *, chunk_size: int, overlap: int) -> Iterator[str]:
    buf = ""
    step = max(1, chunk_size - overlap)
    while True:
        part = stream.read(64 * 1024)
        if not part:
            break
        buf += part
        while len(buf) >= chunk_size:
            yield buf[:chunk_size]
            buf = buf[step:]

    tail = buf.strip()
    if tail:
        yield tail


def _ocr_pdf_to_txt(pdf_path: Path, out_txt_path: Path) -> Tuple[bool, str]:
    """OCR PDF -> TXT. Возвращает (ok, message). Пишет TXT рядом с PDF."""
    try:
        import pymupdf
    except Exception as e:
        return False, f"pymupdf не установлен: {e}"

    try:
        import pytesseract
        from PIL import Image
    except Exception as e:
        return False, f"pytesseract/PIL не доступны: {e}"

    try:
        doc = pymupdf.open(str(pdf_path))
    except Exception as e:
        return False, f"Не удалось открыть PDF: {e}"

    parts: List[str] = []
    try:
        for page_num in range(doc.page_count):
            page = doc.load_page(page_num)
            pix = page.get_pixmap(dpi=300)
            img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
            text = pytesseract.image_to_string(img, lang=os.getenv("OCR_LANG", "rus+eng"))
            text = _normalize_text_player(text)
            if text:
                parts.append(text)
            if page_num % 5 == 0:
                gc.collect()
    except Exception as e:
        return False, f"OCR ошибка: {e}"
    finally:
        doc.close()

    full = "\n\n".join(parts).strip()
    if not full:
        return False, "OCR не извлёк текст (пусто)"

    try:
        out_txt_path.write_text(full, encoding="utf-8", errors="ignore")
    except Exception as e:
        return False, f"Не удалось сохранить TXT: {e}"

    return True, f"OK: {out_txt_path.name}"


class PlayerKnowledgeBase:
    def __init__(self, dataset_dir: Path = PLAYER_DATASET_DIR, faiss_dir: Path = PLAYER_FAISS_DIR, embed_model: Optional[str] = None):
        self.dataset_dir = Path(dataset_dir)
        self.faiss_dir = Path(faiss_dir)
        self.embed_model = embed_model or PLAYER_EMBED_MODEL
        self._vectorstore = None
        self._embeddings = None
        self._splitter = None
        self.last_status = "init"
        self.faiss_dir.mkdir(parents=True, exist_ok=True)

    def _get_embeddings(self):
        if self._embeddings is None:
            from langchain_community.embeddings import HuggingFaceEmbeddings
            device = "cuda" if os.getenv("USE_CUDA", "0") == "1" else "cpu"
            self._embeddings = HuggingFaceEmbeddings(
                model_name=self.embed_model,
                model_kwargs={"device": device},
                encode_kwargs={"normalize_embeddings": True},
            )
            logger.info("Player embeddings initialized (%s) on %s", self.embed_model, device)
        return self._embeddings

    def _reset_embeddings(self, embed_model: str):
        self.embed_model = embed_model
        self._embeddings = None

    def _get_splitter(self):
        if self._splitter is None:
            from langchain_text_splitters import RecursiveCharacterTextSplitter
            self._splitter = RecursiveCharacterTextSplitter(
                chunk_size=PLAYER_CHUNK_SIZE,
                chunk_overlap=PLAYER_CHUNK_OVERLAP,
                separators=["\n\n", "\n", " ", ""],
            )
        return self._splitter

    def _load_vectorstore_if_exists(self):
        if self._vectorstore is not None:
            return self._vectorstore

        index_file = self.faiss_dir / "index.faiss"
        if not index_file.exists():
            return None

        from langchain_community.vectorstores import FAISS

        try:
            self._vectorstore = FAISS.load_local(
                str(self.faiss_dir),
                self._get_embeddings(),
                allow_dangerous_deserialization=True,
            )
            logger.info("Player FAISS index loaded from %s", self.faiss_dir)
            return self._vectorstore
        except Exception as e:
            logger.exception("Player FAISS load failed with %s", self.embed_model)

        # Fallback: если индекс собран другой моделью (например mpnet), пробуем ею
        try:
            self._reset_embeddings(PLAYER_FALLBACK_EMBED_MODEL)
            self._vectorstore = FAISS.load_local(
                str(self.faiss_dir),
                self._get_embeddings(),
                allow_dangerous_deserialization=True,
            )
            logger.info("Player FAISS index loaded with fallback model (%s)", self.embed_model)
            return self._vectorstore
        except Exception:
            logger.exception("Player FAISS load failed even with fallback model")
            self._vectorstore = None
            return None

    def _save_vectorstore(self):
        if self._vectorstore is not None:
            self._vectorstore.save_local(str(self.faiss_dir))

    def _iter_source_files(self) -> List[Path]:
        if not self.dataset_dir.exists():
            return []
        files: List[Path] = []
        for fp in sorted(self.dataset_dir.rglob("*")):
            if not fp.is_file():
                continue
            if fp.suffix.lower() in (".txt", ".md", ".rst", ".csv", ".pdf"):
                files.append(fp)
        return files

    def ocr_pdfs_to_txt(self) -> str:
        if not self.dataset_dir.exists():
            return f"Dataset не найден: {self.dataset_dir}"

        pdfs = [p for p in self._iter_source_files() if p.suffix.lower() == ".pdf"]
        if not pdfs:
            return "PDF не найдено для OCR."

        ok = 0
        fail = 0
        notes = []
        for pdf in pdfs:
            out_txt = pdf.with_suffix(".txt")
            if out_txt.exists() and out_txt.stat().st_size > 200:
                continue
            done, msg = _ocr_pdf_to_txt(pdf, out_txt)
            if done:
                ok += 1
            else:
                fail += 1
                notes.append(f"{pdf.name}: {msg}")

        suffix = "" if not notes else ("\n" + "\n".join(notes[:10]))
        return f"OCR готово. Успешно: {ok}, ошибок: {fail}." + suffix

    def build_or_rebuild_index(self, *, force: bool = False) -> str:
        from langchain_community.vectorstores import FAISS

        self.last_status = "build_start"

        if force:
            self._vectorstore = None
            for fp in self.faiss_dir.glob("*"):
                try:
                    fp.unlink()
                except Exception:
                    pass

        self.ocr_pdfs_to_txt()

        txts = [p for p in self._iter_source_files() if p.suffix.lower() in (".txt", ".md", ".rst", ".csv")]
        if not txts:
            self.last_status = "dataset_empty"
            return f"Нет текстовых файлов для индексации в {self.dataset_dir}"

        splitter = self._get_splitter()
        docs_total = 0
        files_used = 0
        files_skipped = 0
        vs = None

        for i, fp in enumerate(txts):
            try:
                size = fp.stat().st_size
            except Exception:
                size = 0

            try:
                if size >= PLAYER_STREAM_FILE_BYTES:
                    with fp.open("r", encoding="utf-8", errors="ignore") as f:
                        for chunk_id, chunk in enumerate(_iter_char_chunks_from_stream(f, chunk_size=PLAYER_CHUNK_SIZE, overlap=PLAYER_CHUNK_OVERLAP)):
                            text = _normalize_text_player(chunk)
                            if not text or len(text) < 50:
                                continue
                            docs = splitter.create_documents(
                                [text],
                                metadatas=[{"source_file": fp.name, "rel_path": str(fp.relative_to(self.dataset_dir)), "chunk_id": chunk_id}],
                            )
                            if not docs:
                                continue
                            if vs is None:
                                vs = FAISS.from_documents(docs, self._get_embeddings())
                            else:
                                vs.add_documents(docs)
                            docs_total += len(docs)
                            if docs_total % 200 == 0:
                                gc.collect()
                    files_used += 1
                else:
                    raw = fp.read_text(encoding="utf-8", errors="ignore")
                    text = _normalize_text_player(raw)
                    if not text or len(text) < 50:
                        files_skipped += 1
                        continue
                    docs = splitter.create_documents([text], metadatas=[{"source_file": fp.name, "rel_path": str(fp.relative_to(self.dataset_dir))}])
                    if not docs:
                        files_skipped += 1
                        continue
                    if vs is None:
                        vs = FAISS.from_documents(docs, self._get_embeddings())
                    else:
                        vs.add_documents(docs)
                    docs_total += len(docs)
                    files_used += 1

                if i % 5 == 0:
                    gc.collect()

            except Exception as e:
                files_skipped += 1
                logger.exception("Player index build: failed for %s", fp)
                self.last_status = f"build_error:{fp.name}:{e}"

        if vs is None or docs_total == 0:
            self.last_status = "index_empty"
            return "Не удалось создать индекс: нет данных после обработки"

        self._vectorstore = vs
        self._save_vectorstore()
        self.last_status = "ready"
        return f"Player индекс построен: {docs_total} фрагментов. Файлов использовано: {files_used}, пропущено: {files_skipped}. Путь: {self.faiss_dir}"

    def search(self, query: str, *, top_k: int = PLAYER_TOP_K) -> Tuple[bool, str]:
        self.last_status = "search"
        vs = self._load_vectorstore_if_exists()
        if vs is None:
            self.last_status = "index_missing"
            return False, ""

        try:
            ntotal = int(getattr(vs, "index").ntotal)
        except Exception:
            ntotal = None

        if ntotal == 0:
            self.last_status = "index_empty"
            return False, ""

        docs = vs.similarity_search(query, k=top_k)
        if not docs:
            self.last_status = "no_docs"
            return False, ""

        contexts = []
        for d in docs:
            meta = d.metadata or {}
            src = meta.get("rel_path") or meta.get("source_file") or "unknown"
            body = (d.page_content or "").strip()
            if not body:
                continue
            contexts.append(f"[{src}]\n" + body)

        if not contexts:
            self.last_status = "no_text"
            return False, ""

        self.last_status = "found"
        return True, "\n\n---\n\n".join(contexts)


player_kb = PlayerKnowledgeBase()


In [ ]:
# FAISS Telegram bot (compact). Run this cell.
import os, re, time, json, asyncio, logging
from pathlib import Path
from typing import Optional

import nest_asyncio
nest_asyncio.apply()

from telegram import ReplyKeyboardMarkup, KeyboardButton, Update
from telegram.ext import ApplicationBuilder, ContextTypes, MessageHandler, CommandHandler, filters

logging.basicConfig(level=logging.INFO, format="%(asctime)s %(levelname)s %(message)s")
logger = logging.getLogger("rag-dnd-bot")

# ----------------- Config -----------------
TG_BOT_TOKEN = os.getenv("TG_BOT_TOKEN")
OLLAMA_API_URL = os.getenv("OLLAMA_API_URL", "https://ollama.com/api/chat")
OLLAMA_API_KEY = os.getenv("OLLAMA_API_KEY")
OLLAMA_MODEL = os.getenv("OLLAMA_MODEL", "gpt-oss:20b-cloud")

OLLAMA_TEMPERATURE = float(os.getenv("OLLAMA_TEMPERATURE", "0.6"))
OLLAMA_TEMPERATURE_REPLIES = float(os.getenv("OLLAMA_TEMPERATURE_REPLIES", "0.2"))

OLLAMA_NUM_PREDICT = int(os.getenv("OLLAMA_NUM_PREDICT", "900"))
OLLAMA_NUM_PREDICT_REPLIES = int(os.getenv("OLLAMA_NUM_PREDICT_REPLIES", "450"))

TELEGRAM_MAX_CHARS = int(os.getenv("TELEGRAM_MAX_CHARS", "3800"))

FAISS_DIR = Path(os.getenv("FAISS_DIR", "artifacts/faiss_campaign_knowledge"))

VOICE_MALE = "male"
VOICE_FEMALE = "female"

# ----------------- Keyboards -----------------
KB_MAIN = ReplyKeyboardMarkup(
    [[KeyboardButton("Dungeon Master 🧙‍♂️"), KeyboardButton("Player 🎲"), KeyboardButton("Restart 🔄")]],
    resize_keyboard=True,
)
KB_PLAYER = ReplyKeyboardMarkup(
    [[KeyboardButton("Вопросы по правилам"), KeyboardButton("Помощь в ситуации")], [KeyboardButton("← Главное меню")]],
    resize_keyboard=True,
)
KB_DM = ReplyKeyboardMarkup(
    [
        [KeyboardButton("Хранилище знаний"), KeyboardButton("Создание персонажа")],
        [KeyboardButton("Реплики"), KeyboardButton("Боевка")],
        [KeyboardButton("Загрузить информацию")],
        [KeyboardButton("← Главное меню")],
    ],
    resize_keyboard=True,
)
KB_REPLIES = ReplyKeyboardMarkup(
    [[KeyboardButton("Текст"), KeyboardButton("Озвучка + текст")], [KeyboardButton("← Назад (DM)")]],
    resize_keyboard=True,
)
KB_VOICE = ReplyKeyboardMarkup(
    [[KeyboardButton("Мужской"), KeyboardButton("Женский")], [KeyboardButton("← Назад (DM)")]],
    resize_keyboard=True,
)

WELCOME_TEXT = "🎲 RAG DnD Assistant\nВыбери роль."

# ----------------- Helpers -----------------
def normalize_text(text: str) -> str:
    text = (text or "").replace("\r\n", "\n").replace("\r", "\n")
    text = re.sub(r"\n{3,}", "\n\n", text)
    return text.strip()


def chunk_for_tg(text: str, max_chars: int = TELEGRAM_MAX_CHARS):
    t = (text or "").strip()
    if not t:
        return [""]
    out = []
    while t:
        if len(t) <= max_chars:
            out.append(t)
            break
        cut = t.rfind("\n\n", 0, max_chars)
        if cut < max_chars * 0.4:
            cut = t.rfind("\n", 0, max_chars)
        if cut < max_chars * 0.4:
            cut = t.rfind(" ", 0, max_chars)
        if cut < max_chars * 0.4:
            cut = max_chars
        out.append(t[:cut].rstrip())
        t = t[cut:].lstrip()
    return out


async def safe_reply(update: Update, text: str, *, reply_markup=None):
    for i, part in enumerate(chunk_for_tg(text)):
        if i == 0:
            await update.message.reply_text(part, reply_markup=reply_markup)
        else:
            await update.message.reply_text(part)


def _looks_like_bad_reply(text: str) -> bool:
    t = normalize_text(text)
    if not t:
        return True
    if len(t) > 700:
        return True
    low = t.lower()
    bad_words = ["зашквар", "кринж", "рофл", "лол", "мем", "интернет", "смартфон", "компьютер", "лазер", "ракета", "электрич"]
    return any(w in low for w in bad_words)


def _player_warning(found: bool) -> str:
    if found:
        return ""
    return "Я не нашёл подходящих фрагментов в базе знаний игрока; ответ ниже — по общим знаниям D&D 5e."


# ----------------- Ollama -----------------
class OllamaCloud:
    def __init__(self):
        self.url = OLLAMA_API_URL
        self.model = OLLAMA_MODEL
        self.key = OLLAMA_API_KEY

    def chat(self, messages, *, num_predict: int, temperature: float, timeout: int = 120) -> str:
        import requests

        headers = {"Content-Type": "application/json"}
        if self.key:
            headers["Authorization"] = "Bearer " + self.key

        body = {
            "model": self.model,
            "messages": messages,
            "stream": False,
            "options": {"num_predict": int(num_predict), "temperature": float(temperature)},
        }
        r = requests.post(self.url, json=body, headers=headers, timeout=timeout)
        r.raise_for_status()
        data = r.json() if r.content else {}
        msg = (data.get("message") or {})
        content = msg.get("content")
        if isinstance(content, str) and content.strip():
            return content.strip()
        resp = data.get("response")
        if isinstance(resp, str) and resp.strip():
            return resp.strip()
        return ""


def ollama_with_retries(ollama: OllamaCloud, messages, *, num_predict: int, temperature: float, retries: int = 2) -> str:
    for i in range(retries + 1):
        try:
            out = (ollama.chat(messages, num_predict=num_predict, temperature=temperature) or "").strip()
            if out:
                return out
        except Exception:
            pass
        time.sleep(0.6 * (i + 1))
    return ""


ollama = OllamaCloud()

# ----------------- DM campaign retrieval -----------------
class CampaignKB:
    def __init__(self, faiss_dir: Path):
        self.faiss_dir = Path(faiss_dir)
        self._vs = None
        self._emb = None

    def _embeddings(self):
        if self._emb is None:
            from langchain_community.embeddings import HuggingFaceEmbeddings
            device = "cuda" if os.getenv("USE_CUDA", "0") == "1" else "cpu"
            self._emb = HuggingFaceEmbeddings(
                model_name=os.getenv("SAFE_EMBED_MODEL", "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"),
                model_kwargs={"device": device},
                encode_kwargs={"normalize_embeddings": True},
            )
        return self._emb

    def _load(self):
        if self._vs is not None:
            return self._vs
        if not (self.faiss_dir / "index.faiss").exists():
            return None
        from langchain_community.vectorstores import FAISS
        self._vs = FAISS.load_local(str(self.faiss_dir), self._embeddings(), allow_dangerous_deserialization=True)
        return self._vs

    def retrieve(self, query: str, k: int = 6) -> str:
        vs = self._load()
        if vs is None:
            return ""
        docs = vs.similarity_search(query, k=k)
        parts = []
        for d in docs:
            meta = d.metadata or {}
            src = meta.get("source_file") or "unknown"
            page = meta.get("page")
            pfx = f"[{src}{', стр. ' + str(page) if page else ''}]\n"
            txt = (d.page_content or "").strip()
            if txt:
                parts.append(pfx + txt)
        return "\n\n---\n\n".join(parts).strip()


campaign_kb = CampaignKB(FAISS_DIR)

# ----------------- Player KB -----------------
# player_kb is created in previous cell (PlayerKnowledgeBase)

def player_retrieve(query: str, k: int = 15) -> tuple[bool, str]:
    try:
        return player_kb.search(query, top_k=k)  # type: ignore
    except Exception:
        return False, ""


def player_answer(prompt_template: str, user_text: str) -> str:
    found, ctx = player_retrieve(user_text, k=15)
    warn = _player_warning(found)

    prompt = prompt_template + "\n\nФрагменты из базы знаний игрока:\n" + (ctx if ctx else "(нет)") + "\n\nЗапрос игрока:\n" + (user_text or "")

    msgs = [
        {"role": "system", "content": "Вывод только простым текстом. Без Markdown/JSON/таблиц."},
        {"role": "user", "content": prompt},
    ]

    out = ollama_with_retries(ollama, msgs, num_predict=OLLAMA_NUM_PREDICT, temperature=OLLAMA_TEMPERATURE, retries=2)
    out = normalize_text(out)
    if not out:
        return "Сервис ответа временно не вернул текст. Попробуй ещё раз."

    return (warn + "\n\n" + out).strip() if warn else out


# ----------------- TTS (Silero) -----------------
class SileroTTS:
    def __init__(self):
        self.model = None
        self.sample_rate = 48000
        self.speakers = {"Мужской": "aidar", "Женский": "xenia", VOICE_MALE: "aidar", VOICE_FEMALE: "xenia"}

    def _init(self):
        if self.model is not None:
            return
        import torch
        self.model, _ = torch.hub.load(repo_or_dir="snakers4/silero-models", model="silero_tts", language="ru", speaker="v5_ru")
        self.model.to(torch.device("cpu"))

    def synthesize(self, text: str, voice: str, out_dir: Path) -> Path:
        import numpy as np
        self._init()
        speaker = self.speakers.get(voice, "aidar")
        out_dir.mkdir(parents=True, exist_ok=True)
        base = f"tts_{speaker}_{int(time.time()*1000)}"
        wav = out_dir / f"{base}.wav"
        audio = self.model.apply_tts(text=(text or "").strip(), speaker=speaker, sample_rate=self.sample_rate, put_accent=True, put_yo=True)
        audio = np.asarray(audio)
        try:
            import soundfile as sf
            sf.write(str(wav), audio, self.sample_rate)
        except Exception:
            from scipy.io.wavfile import write as w
            max_abs = float(np.max(np.abs(audio))) or 1.0
            w(str(wav), self.sample_rate, (audio / max_abs * 32767).astype(np.int16))
        try:
            from pydub import AudioSegment
            mp3 = out_dir / f"{base}.mp3"
            AudioSegment.from_wav(str(wav)).export(str(mp3), format="mp3")
            return mp3
        except Exception:
            return wav


tts = SileroTTS()

# ----------------- NPC replies -----------------
NPC_DIALOG_SYSTEM = NPC_DIALOG_PROMPT


def gen_npc_reply(user_prompt: str) -> str:
    ctx = campaign_kb.retrieve(user_prompt, k=6)
    payload = "Контекст кампании:\n" + (ctx if ctx else "(нет)") + "\n\nСитуация/реплика игрока:\n" + (user_prompt or "")
    msgs = [{"role": "system", "content": NPC_DIALOG_SYSTEM}, {"role": "user", "content": payload}]
    out = ollama_with_retries(ollama, msgs, num_predict=OLLAMA_NUM_PREDICT_REPLIES, temperature=OLLAMA_TEMPERATURE_REPLIES, retries=2)
    out = normalize_text(out)
    if not out or _looks_like_bad_reply(out):
        msgs2 = [
            {"role": "system", "content": "Перепиши как одну связную реплику NPC (1–3 предложения), хороший русский, без сленга и без современных терминов."},
            {"role": "user", "content": payload + "\n\nЧерновик:\n" + (out or "")},
        ]
        out2 = normalize_text(ollama_with_retries(ollama, msgs2, num_predict=OLLAMA_NUM_PREDICT_REPLIES, temperature=0.1, retries=2))
        if out2:
            out = out2
    return out or "Сервис ответа временно не вернул текст. Попробуй ещё раз."


# ----------------- Bot handlers -----------------
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    context.user_data.clear()
    await safe_reply(update, WELCOME_TEXT, reply_markup=KB_MAIN)


async def router(update: Update, context: ContextTypes.DEFAULT_TYPE):
    text = (update.message.text or "").strip()

    # Global navigation
    if text == "Restart 🔄":
        context.user_data.clear()
        await safe_reply(update, "Контекст очищен.", reply_markup=KB_MAIN)
        return
    if text == "← Главное меню":
        context.user_data.clear()
        await safe_reply(update, WELCOME_TEXT, reply_markup=KB_MAIN)
        return

    # Role selection (repeatable)
    if text == "Dungeon Master 🧙‍♂️":
        context.user_data["role"] = "dm"
        context.user_data["mode"] = None
        await safe_reply(update, "Меню мастера:", reply_markup=KB_DM)
        return
    if text == "Player 🎲":
        context.user_data["role"] = "player"
        context.user_data["mode"] = None
        await safe_reply(update, "Меню игрока:", reply_markup=KB_PLAYER)
        return

    role = context.user_data.get("role")
    mode = context.user_data.get("mode")

    # Player menus
    if role == "player":
        if text == "Вопросы по правилам":
            context.user_data["mode"] = "player_rules"
            await safe_reply(update, "Задай вопрос по правилам:")
            return
        if text == "Помощь в ситуации":
            context.user_data["mode"] = "player_situation"
            await safe_reply(update, "Опиши ситуацию:")
            return

        if mode == "player_rules":
            await safe_reply(update, player_answer(PLAYER_RULES_PROMPT, text))
            return
        if mode == "player_situation":
            await safe_reply(update, player_answer(PLAYER_SITUATION_PROMPT, text))
            return

        await safe_reply(update, "Меню игрока:", reply_markup=KB_PLAYER)
        return

    # DM menus
    if role == "dm":
        if text == "Реплики":
            context.user_data["mode"] = "replies_menu"
            await safe_reply(update, "Формат:", reply_markup=KB_REPLIES)
            return
        if text == "Текст":
            context.user_data["mode"] = "npc_text"
            await safe_reply(update, "Промпт для реплики:")
            return
        if text == "Озвучка + текст":
            context.user_data["mode"] = "npc_tts_voice"
            await safe_reply(update, "Выбери голос NPC:", reply_markup=KB_VOICE)
            return
        if text == "Мужской" and mode == "npc_tts_voice":
            context.user_data["tts_voice"] = VOICE_MALE
            context.user_data["mode"] = "npc_tts"
            await safe_reply(update, "Отправь промпт для реплики NPC (верну текст + аудио).", reply_markup=KB_REPLIES)
            return
        if text == "Женский" and mode == "npc_tts_voice":
            context.user_data["tts_voice"] = VOICE_FEMALE
            context.user_data["mode"] = "npc_tts"
            await safe_reply(update, "Отправь промпт для реплики NPC (верну текст + аудио).", reply_markup=KB_REPLIES)
            return

        mode = context.user_data.get("mode")

        if mode == "npc_text":
            await safe_reply(update, gen_npc_reply(text))
            return

        if mode == "npc_tts":
            reply = gen_npc_reply(text)
            await safe_reply(update, reply)

            voice = context.user_data.get("tts_voice", VOICE_MALE)
            out_dir = Path("artifacts/tts")
            loop = asyncio.get_running_loop()
            try:
                audio_path = await loop.run_in_executor(None, lambda: tts.synthesize(reply, voice, out_dir))
                with open(audio_path, "rb") as f:
                    await update.message.reply_audio(audio=f, filename=audio_path.name)
            except Exception as e:
                logger.exception("TTS error")
                await safe_reply(update, f"Не удалось озвучить: {e}")
            return

        if text == "← Назад (DM)":
            context.user_data["mode"] = None
            await safe_reply(update, "Меню DM:", reply_markup=KB_DM)
            return

        if text == "Хранилище знаний":
            context.user_data["mode"] = "dm_knowledge"
            await safe_reply(update, "Запрос по кампании:")
            return

        if mode == "dm_knowledge":
            ctx = campaign_kb.retrieve(text, k=6)
            if not ctx:
                await safe_reply(update, "Ничего не найдено в хранилище.")
                return
            msgs = [
                {"role": "system", "content": "Отвечай только по предоставленному контексту. Простым текстом."},
                {"role": "user", "content": ctx + "\n\nВопрос: " + text},
            ]
            out = normalize_text(ollama_with_retries(ollama, msgs, num_predict=OLLAMA_NUM_PREDICT, temperature=OLLAMA_TEMPERATURE, retries=2))
            await safe_reply(update, out or "Сервис ответа временно не вернул текст. Попробуй ещё раз.")
            return

        await safe_reply(update, "Меню DM:", reply_markup=KB_DM)
        return

    # No role chosen yet
    await safe_reply(update, "Выбери роль:", reply_markup=KB_MAIN)


def main():
    if not TG_BOT_TOKEN:
        raise RuntimeError("TG_BOT_TOKEN не задан")

    app = ApplicationBuilder().token(TG_BOT_TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, router))

    logger.info("Bot started")
    app.run_polling(close_loop=False)


if __name__ == "__main__":
    main()


2026-01-14 20:17:10,467 INFO Bot started
2026-01-14 20:17:10,867 INFO HTTP Request: POST https://api.telegram.org/bot1/getMe "HTTP/1.1 200 OK"
2026-01-14 20:17:10,942 INFO HTTP Request: POST https://api.telegram.org/bot1/deleteWebhook "HTTP/1.1 200 OK"
2026-01-14 20:17:10,945 INFO Application started
2026-01-14 20:17:11,300 INFO HTTP Request: POST https://api.telegram.org/bot1/getUpdates "HTTP/1.1 200 OK"
2026-01-14 20:17:11,469 INFO HTTP Request: POST https://api.telegram.org/bot1/sendMessage "HTTP/1.1 200 OK"
2026-01-14 20:17:15,606 INFO HTTP Request: POST https://api.telegram.org/bot1/getUpdates "HTTP/1.1 200 OK"
2026-01-14 20:17:15,724 INFO HTTP Request: POST https://api.telegram.org/bot1/sendMessage "HTTP/1.1 200 OK"
2026-01-14 20:17:16,644 INFO HTTP Request: POST https://api.telegram.org/bot1/getUpdates "HTTP/1.1 200 OK"
2026-01-14 20:17:16,758 INFO HTTP Request: POST https://api.telegram.org/bot1/sendMessage "HTTP/1.1 200 OK"
2026-01-14 20:17:18,301 INFO HTTP Request: POST https